In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

### **Data Reading From Source**

In [0]:
df = spark.sql("select * from databricks_cata.silver.customers_silver")

**Removing Duplicates**

In [0]:
df = df.dropDuplicates(subset=['customer_id'])

In [0]:
df.limit(10).display()

customer_id,email,city,state,domains,full_name
C01220,matthew01@yahoo.com,New Andrewhaven,WA,yahoo.com,Thomas Fitzgerald
C01579,nathancastro@gmail.com,South Amanda,FL,gmail.com,Brittany Schmidt
C01155,cynthia51@lewis-dixon.com,East Theresa,FL,lewis-dixon.com,Latasha Phelps
C01943,rodriguezzachary@hotmail.com,East Timothy,PA,hotmail.com,David Cooper
C01554,sthompson@harding.com,New Leefurt,ME,harding.com,Gavin Lindsey
C00767,rsmith@pruitt-hodges.net,Heatherton,IL,pruitt-hodges.net,Pam Watts
C01097,joel22@stone-holmes.com,Davidhaven,LA,stone-holmes.com,James Martinez
C01674,crystalraymond@keller.com,Hoborough,ME,keller.com,Nicole Johnston
C00215,petersonthomas@yahoo.com,South Andrea,OK,yahoo.com,Danielle Huerta
C00587,phillipsstephanie@gmail.com,New Ronaldmouth,OR,gmail.com,Michael Bailey


#  **Dividing New vs Old Records**

In [0]:
if (spark.catalog.tableExists("databricks_cata.gold.DimCustomers")):

    df_old = spark.sql('''select DimCustomerKey, customer_id, create_date, update_date 
                       from databricks_cata.gold.DimCustomers''')
    
else: 

    df_old = spark.sql('''select 0 DimCustomerKey, 0 customer_id, 0 create_date, 0 update_date 
                        FROM databricks_cata.silver.customers_silver where 1=0''')

In [0]:
df_old.display()

**Reanming Columns of df_old**

In [0]:
df_old = df_old.withColumnRenamed("DimCustomerKey", "old_DimCustomerKey")\
                    .withColumnRenamed("customer_id", "old_customer_id")\
                    .withColumnRenamed("create_date", "old_create_date")\
                    .withColumnRenamed("update_date", "old_update_date")

**Applying Join with the Old Records**

In [0]:
df_join = df.join(df_old, df['customer_id'] == df_old['old_customer_id'], 'left')

In [0]:
df_join.display()

**Seperating New vs Old Records**

In [0]:
df_new = df_join.filter(df_join['old_DimCustomerKey'].isNull())

In [0]:
df_old = df_join.filter(df_join['old_DimCustomerKey'].isNotNull())

**Preparing df_old**

In [0]:
# Dropping all the columns which are not required

df_old = df_old.drop('old_customer_id','old_update_date')

# Renaming "old_DimCustomerKey" to "DimCustomerKey"

df_old = df_old.withColumnRenamed("old_DimCustomerKey", "DimCustomerKey")

# Renaming "old_create_date" column to "create_date"

df_old = df_old.withColumnRenamed("old_create_date", "create_date")
df_old = df_old.withColumn("create_date",to_timestamp(col("create_date")))

# Recreating "update_date" column with current timestamp

df_old = df_old.withColumn("update_date", current_timestamp())

In [0]:
df_old.display()

**Preparing df_new**

In [0]:
# Dropping all the columns which are not required

df_new = df_new.drop('old_DimCustomerKey', 'old_customer_id','old_update_date','old_create_date')

# Recreating "update_date", "current_date" columns with current timestamp

df_new = df_new.withColumn("update_date", current_timestamp())
df_new = df_new.withColumn("create_date", current_timestamp())

In [0]:
df_new.display()

customer_id,email,city,state,domains,full_name,update_date,create_date


**Surrogate Key - From 1**

In [0]:
df_new = df_new.withColumn("DimCustomerKey",monotonically_increasing_id()+lit(1)) 

In [0]:
df_new.display()

customer_id,email,city,state,domains,full_name,update_date,create_date,DimCustomerKey


**Adding Max Surrogate Key**

In [0]:
if (spark.catalog.tableExists("databricks_cata.gold.DimCustomers")):
    max_surrogate_key = 0

else: 
  df_maxsur = spark.sql("select max(DimCustomerKey) as max_surrogate_key from databricks_cata.gold.DimCustomers")

  #Converting df_maxsur to max_surrogate_key
  max_surrogate_key = df_maxsur.collect()[0]['max_surrogate_key']


In [0]:
df_new = df_new.withColumn("DimCustomerKey",lit(max_surrogate_key)+col("DimCustomerKey"))

**Union of df_old and df_new**

In [0]:
df_final = df_new.unionByName(df_old)

In [0]:
df_final.display()

## **SCD Type - 1**

In [0]:
from delta.tables import DeltaTable

In [0]:
if (spark.catalog.tableExists("databricks_cata.gold.DimCustomers")):
    
    dlt_obj = DeltaTable.forPath(spark,"abfss://gold@databricksete.dfs.core.windows.net/DimCustomers")

    dlt_obj.alias("trg").merge(df_final.alias("src"),"trg.DimCustomerKey = src.DimCustomerKey")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

else:
    df_final.write.mode("overwrite")\
    .format("delta")\
    .option("path","abfss://gold@databricksete.dfs.core.windows.net/DimCustomers")\
    .saveAsTable("databricks_cata.gold.DimCustomers")


In [0]:
%sql
-- SELECT * FROM databricks_cata.gold.dimcustomers